<a href="https://colab.research.google.com/github/doorqman/quant.invest/blob/master/%E5%89%B5%E6%96%B0%E9%AB%98%E5%9F%BA%E6%9C%AC%E9%9D%A2%E7%AD%96%E7%95%A5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from finlab.data import Data

data = Data()


#############
# 股價淨值比  #
#############
price = data.get('收盤價', 300)
股東權益 = data.get('歸屬於母公司業主之權益合計', 1)
股本 = data.get('普通股股本', 1)


流通股數 = 股本 / 10
每股淨值 = 股東權益 / 流通股數
今日股價 = price.iloc[-1]

股價淨值比pb = (今日股價 / 每股淨值).squeeze()

#####################
# 股東權益報酬率 ROE  #
#####################

稅後淨利 = data.get('本期淨利（淨損）', 2)

# 股東權益，有兩個名稱，有些公司叫做權益總計，有些叫做權益總額
# 所以得把它們抓出來
權益總計 = data.get('權益總計', 2)
權益總額 = data.get('權益總額', 2)

# 並且把它們合併起來
權益總計.fillna(權益總額, inplace=True)

股東權益報酬率 = 稅後淨利.iloc[-1] / 權益總計.iloc[-1]
股東權益報酬率_上季 = 稅後淨利.iloc[-2] / 權益總計.iloc[-2]


#####################
# 成交量  #
#####################
vol = data.get('成交股數', 100)


############
# 篩選股票  #
############

condition1 = (price.rolling(60).max() == price).iloc[-10:].sum() > 0
condition2 = (股東權益報酬率 > 0.03) & (股東權益報酬率_上季 > 0.01)
condition3 = (price.iloc[-1] > 7) & (vol.iloc[-1] > 100000)
rank = 股東權益報酬率 / 股東權益報酬率_上季 / 股價淨值比pb * (condition1 & condition2 & condition3)

print('近10天創新高的股票數量')
print(condition1.sum())

print('rank 前10名的股票')
print(rank[rank > 0].nlargest(10))


### 交易方法

* **入場：假如上述清單有股票，則加入portfolio**

* **出場：跌破20日線或60日線，隔天出場**


### 再平衡方法

文章中的回測是每10天換一次股票，